In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import math

In [2]:
pwd

'/Users/amudide/REVISIONS'

# Global

In [3]:
raw = pd.read_csv('raw-data/hetmat_human200_179sp.txt', sep = '\t')
cols = []
for i in range(len(raw)):
    cols.append(str(raw.loc[i][0]) + ' ' + str(raw.loc[i][1]) + ' ' + str(raw.loc[i][2]))

In [4]:
iucn = pd.read_csv("raw-data/zoonomia_dataset_17nov2021_streamlined.csv", index_col = 0)

In [5]:
iucn

,Order,Family,IUCN,threatened,wild_status_ref,wild_status_reseq,diet_hoc,meanGQ,meanDP,Source,...,froh,ppn_conserved,phylop_kurtosis,ppn_miss_conserved,het_miss_impc_L,het_miss_impc_V,het_lof_impc_L,het_lof_impc_V,hom_miss_impc_L,hom_miss_impc_V
Sp,,,,,,,,,,,,,,,,,,,,,
Acinonyx_jubatus,CARNIVORA,Felidae,VU,1.0,wild,wild,Carnivore,48.557864,5.734421,2. Existing assembly,...,0.868654,0.033622,23.531877,0.131824,NaN,NaN,NaN,NaN,0.308549,0.378047
Acomys_cahirinus,RODENTIA,Muridae,LC,0.0,captive,captive,Omnivore,74.006180,28.037083,1. Zoonomia,...,0.957857,0.036393,20.859896,0.069992,NaN,NaN,NaN,NaN,0.237946,0.315492
Ailuropoda_melanoleuca,CARNIVORA,Ursidae,VU,1.0,captive,captive,Omnivore,91.852011,15.341954,2. Existing assembly,...,0.337415,0.029873,22.839135,0.087625,0.323529,0.296296,0.000000,0.000000,0.254025,0.348917
Ailurus_fulgens,CARNIVORA,Ailuridae,EN,1.0,wild,wild,Omnivore,NaN,NaN,2. Existing assembly,...,NaN,0.022421,26.727905,0.062419,NaN,NaN,NaN,NaN,0.223638,0.318392
Allactaga_bullata,RODENTIA,Dipodidae,LC,0.0,wild,wild,Omnivore,96.613555,24.537520,1. Zoonomia,...,0.027534,0.037799,22.285147,0.066352,0.273006,0.317007,0.000000,0.004082,0.224128,0.293176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vulpes_lagopus,CARNIVORA,Canidae,LC,0.0,captive,captive,Carnivore,93.447795,22.669996,1. Zoonomia,...,0.482580,0.024222,28.676295,0.054605,0.252000,0.309562,0.000000,0.003241,0.194462,0.286502
Xerus_inauris,RODENTIA,Sciuridae,LC,0.0,wild,wild,Omnivore,81.999646,18.393889,1. Zoonomia,...,0.361681,0.029061,23.128070,0.062846,0.353175,0.326625,0.000000,0.000000,0.222449,0.299498
Zalophus_californianus,CARNIVORA,Otariidae,LC,0.0,wild,wild,Carnivore,97.741809,62.983619,1. Zoonomia,...,0.381731,0.028800,25.108636,0.077842,0.307692,0.388186,0.018100,0.002110,0.234710,0.325076


In [6]:
iucn = iucn[['IUCN']]

In [7]:
iucn

,IUCN
Sp,
Acinonyx_jubatus,VU
Acomys_cahirinus,LC
Ailuropoda_melanoleuca,VU
Ailurus_fulgens,EN
Allactaga_bullata,LC
...,...
Vulpes_lagopus,LC
Xerus_inauris,LC
Zalophus_californianus,LC


In [8]:
o = pd.read_csv('raw-data/Coding_Vars_and_phyloP_240sp.txt', sep='\t', index_col=0)
o = o[['coding_vars']]

In [9]:
o

,coding_vars
Sp,
Acinonyx_jubatus,30705
Acomys_cahirinus,580724
Ailuropoda_melanoleuca,163897
Ailurus_fulgens,405816
Allactaga_bullata,213917
...,...
Vulpes_lagopus,78579
Xerus_inauris,342606
Zalophus_californianus,92245


# Functions

In [10]:
'''

Given a raw dataset and a name, accomplish the following

(1) transpose
(2) fix column names
(3) plot + remove NaNs species
(4) fix row names

'''

def read(dataset, name):
    dataset = dataset.T
    try:
        dataset.drop(['chr', 'start', 'stop'], axis=0, inplace = True)
    except:
        pass
    dataset.columns = cols
    
    nanperrow = (dataset.isnull().sum(axis=1).tolist())
    indices = [i for i, x in enumerate(nanperrow) if (x > 30000)]
    dataset.drop(dataset.index[indices], inplace=True)
    
    dataset.index = [i.split('.')[0] for i in dataset.index]
    
    return dataset

In [11]:
'''

Given a dataset, normalize with respect to the the total number of coding variants

'''

def normalize(dataset):
    
    dataset = dataset.drop(['Capra_hircus'], axis=0)
    
    tmp = dataset.T
    for i in tmp.columns:
        tmp[i] /= int(o.loc[i].tolist()[0])
        
    return dataset

In [12]:
'''

Given a list of datasets, merge them into one dataset

'''

def merge(datasets, common):
    
    df = pd.concat([dataset.loc[common] for dataset in datasets], axis=1)
    return df

In [13]:
'''

Given a dataset, add IUCN labels

'''

def addiucn(dataset):

    subset = iucn.loc[dataset.index]
                
    dataset['iucn'] = subset
    
    return dataset

In [14]:
'''

Given a dataset and a name, segregate the non-DD and DD species into two files

'''

def split(df, name):
    
    ndf = df[df['iucn'] != 'DD']
    ddf = df[df['iucn'] == 'DD']
    
    ndf.to_csv('data/' + name + '.csv')
    ddf.to_csv('dd-data/' + name + '-dd' + '.csv')

# Analysis

In [15]:
het = pd.read_csv('raw-data/hetmat_human200_179sp.txt', sep = '\t')
roh = pd.read_csv('raw-data/rohmat_human200_179sp.txt', sep = '\t')
snpphylop = pd.read_csv('raw-data/snpphylopmat_human200_179sp.txt', sep = '\t')
miscons = pd.read_csv('raw-data/missense_conserved_human200_179sp.txt', sep = '\t')
miscount = pd.read_csv('raw-data/missense_counts_human200_179sp.txt', sep = '\t')

In [16]:
het = read(het, 'het')
roh = read(roh, 'roh')
snpphylop = read(snpphylop, 'snpphylop')
miscons = read(miscons, 'miscons')
miscount = read(miscount, 'miscount')

In [17]:
common171 = snpphylop.index.tolist()

In [18]:
len(common171)

171

In [19]:
het = het.loc[common171]
roh = roh.loc[common171]
snpphylop = snpphylop.loc[common171]
miscons = miscons.loc[common171]
miscount = miscount.loc[common171]

In [20]:
miscons = normalize(miscons)
miscount = normalize(miscount)

In [21]:
common170 = miscons.index.tolist()

In [22]:
len(common170)

170

In [23]:
three = merge([het, roh, snpphylop], common171)
five = merge([het, roh, snpphylop, miscons, miscount], common170)

In [24]:
five

,chr1 0 49999,chr1 50000 99999,chr1 100000 149999,chr1 150000 199999,chr1 200000 249999,chr1 250000 299999,chr1 300000 349999,chr1 350000 399999,chr1 400000 449999,chr1 450000 499999,...,chr9 137900000 137949999,chr9 137950000 137999999,chr9 138000000 138049999,chr9 138050000 138099999,chr9 138100000 138149999,chr9 138150000 138199999,chr9 138200000 138249999,chr9 138250000 138299999,chr9 138300000 138349999,chr9 138350000 138399999
Acinonyx_jubatus,0.000131,0.000231,0.000287,0.000225,NaN,NaN,NaN,NaN,NaN,0.000278,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
Acomys_cahirinus,0.000021,0.000005,0.000048,0.000025,NaN,NaN,NaN,NaN,NaN,0.000055,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
Allactaga_bullata,0.003777,NaN,0.003,0.003716,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
Alouatta_palliata,0.001426,0.002579,0.00104,0.001285,NaN,0.000812,NaN,NaN,0.000663,0.001223,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
Ammotragus_lervia,0.000385,0.00107,0.000751,0.000395,NaN,NaN,NaN,NaN,NaN,0.000683,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zalophus_californianus,0.001184,0.001434,0.000624,0.001186,NaN,NaN,NaN,NaN,NaN,0.000717,...,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,0.0,NaN,NaN
Zapus_hudsonius,0.004164,0.005097,0.006403,0.004154,NaN,NaN,NaN,NaN,NaN,0.006989,...,NaN,NaN,0.000015,0.0,0.000034,NaN,NaN,0.0,NaN,NaN
Puma_concolor,0.000582,0.003595,0.001852,0.001068,NaN,NaN,NaN,NaN,NaN,0.000934,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN
Rhinolophus_sinicus,0.003598,NaN,0.006032,0.005158,NaN,NaN,NaN,NaN,NaN,0.004576,...,NaN,NaN,0.000000,0.0,0.000000,NaN,NaN,0.0,NaN,NaN


In [25]:
cols = three.columns.tolist()
for i in range(0, 57509):
    cols[i] = cols[i] + '-het'
for i in range(57509, 57509*2):
    cols[i] = cols[i] + '-roh'
for i in range(57509*2, 57509*3):
    cols[i] = cols[i] + '-snpphylop'
three.columns = cols

In [26]:
cols = five.columns.tolist()
for i in range(0, 57509):
    cols[i] = cols[i] + '-het'
for i in range(57509, 57509*2):
    cols[i] = cols[i] + '-roh'
for i in range(57509*2, 57509*3):
    cols[i] = cols[i] + '-snpphylop'
for i in range(57509*3, 57509*4):
    cols[i] = cols[i] + '-miscons'
for i in range(57509*4, 57509*5):
    cols[i] = cols[i] + '-miscount'
five.columns = cols

In [27]:
het = addiucn(het)
roh = addiucn(roh)
snpphylop = addiucn(snpphylop)
miscons = addiucn(miscons)
miscount = addiucn(miscount)
three = addiucn(three)
five = addiucn(five)

In [28]:
df = pd.read_csv('data/full-summary.csv', index_col = [0])

In [29]:
df

,harm_mean_wt2,gw_het_mean,outbred_het_mode,froh,ppn_conserved,phylop_kurtosis,ppn_miss_conserved,het_miss_impc_L,het_miss_impc_V,het_lof_impc_L,het_lof_impc_V,hom_miss_impc_L,hom_miss_impc_V,iucn
Sp,,,,,,,,,,,,,,
Acinonyx_jubatus,0.077523,0.000335,0.00055,0.868654,0.033622,23.531877,0.131824,NaN,NaN,NaN,NaN,0.308549,0.378047,VU
Acomys_cahirinus,2.539580,0.000121,0.00040,0.957857,0.036393,20.859896,0.069992,NaN,NaN,NaN,NaN,0.237946,0.315492,LC
Ailuropoda_melanoleuca,1.175983,0.004487,0.00100,0.337415,0.029873,22.839135,0.087625,0.323529,0.296296,0.000000,0.000000,0.254025,0.348917,VU
Ailurus_fulgens,1.865817,NaN,NaN,NaN,0.022421,26.727905,0.062419,NaN,NaN,NaN,NaN,0.223638,0.318392,EN
Allactaga_bullata,9.875471,0.002998,0.00280,0.027534,0.037799,22.285147,0.066352,0.273006,0.317007,0.000000,0.004082,0.224128,0.293176,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vulpes_lagopus,3.682468,0.001478,0.00210,0.482580,0.024222,28.676295,0.054605,0.252000,0.309562,0.000000,0.003241,0.194462,0.286502,LC
Xerus_inauris,4.077529,0.001585,0.00060,0.361681,0.029061,23.128070,0.062846,0.353175,0.326625,0.000000,0.000000,0.222449,0.299498,LC
Zalophus_californianus,0.579708,0.000914,0.00060,0.381731,0.028800,25.108636,0.077842,0.307692,0.388186,0.018100,0.002110,0.234710,0.325076,LC


In [30]:
df = df.drop(['iucn'], axis=1)

In [31]:
df

,harm_mean_wt2,gw_het_mean,outbred_het_mode,froh,ppn_conserved,phylop_kurtosis,ppn_miss_conserved,het_miss_impc_L,het_miss_impc_V,het_lof_impc_L,het_lof_impc_V,hom_miss_impc_L,hom_miss_impc_V
Sp,,,,,,,,,,,,,
Acinonyx_jubatus,0.077523,0.000335,0.00055,0.868654,0.033622,23.531877,0.131824,NaN,NaN,NaN,NaN,0.308549,0.378047
Acomys_cahirinus,2.539580,0.000121,0.00040,0.957857,0.036393,20.859896,0.069992,NaN,NaN,NaN,NaN,0.237946,0.315492
Ailuropoda_melanoleuca,1.175983,0.004487,0.00100,0.337415,0.029873,22.839135,0.087625,0.323529,0.296296,0.000000,0.000000,0.254025,0.348917
Ailurus_fulgens,1.865817,NaN,NaN,NaN,0.022421,26.727905,0.062419,NaN,NaN,NaN,NaN,0.223638,0.318392
Allactaga_bullata,9.875471,0.002998,0.00280,0.027534,0.037799,22.285147,0.066352,0.273006,0.317007,0.000000,0.004082,0.224128,0.293176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vulpes_lagopus,3.682468,0.001478,0.00210,0.482580,0.024222,28.676295,0.054605,0.252000,0.309562,0.000000,0.003241,0.194462,0.286502
Xerus_inauris,4.077529,0.001585,0.00060,0.361681,0.029061,23.128070,0.062846,0.353175,0.326625,0.000000,0.000000,0.222449,0.299498
Zalophus_californianus,0.579708,0.000914,0.00060,0.381731,0.028800,25.108636,0.077842,0.307692,0.388186,0.018100,0.002110,0.234710,0.325076


In [32]:
het1 = merge([het, df], common171)
roh1 = merge([roh, df], common171)
snpphylop1 = merge([snpphylop, df], common171)
miscons1 = merge([miscons, df], common170)
miscount1 = merge([miscount, df], common170)
three1 = merge([three, df], common171)
five1 = merge([five, df], common170)

In [33]:
split(het, 'het-170')
split(roh, 'roh-170')
split(snpphylop, 'snpphylop-170')
split(miscons, 'miscons-170')
split(miscount, 'miscount-170')
split(three, 'three-170')
split(five, 'five-170')

In [34]:
split(het1, 'het-summary-170')
split(roh1, 'roh-summary-170')
split(snpphylop1, 'snpphylop-summary-170')
split(miscons1, 'miscons-summary-170')
split(miscount1, 'miscount-summary-170')
split(three1, 'three-summary-170')
split(five1, 'five-summary-170')

In [ ]:
raw.columns.tolist()

In [ ]:
len(raw.columns)

In [ ]:
tmp = pd.read_csv('data/miscons-summary-170.csv')

In [ ]:
tmp

In [ ]:
tmp.index = tmp['Unnamed: 0']

In [ ]:
set(tmp['iucn'])